In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import random
import os
import time
import csv

if not os.path.exists('./restComments1_csv'):
    os.mkdir('./restComments1_csv')

m = 0    
url='https://www.tripadvisor.com.tw/Restaurants-g293913-oa{}-Taipei.html'.format(m*30)

userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0'
headers = {'User-Agent':userAgent}
ss = requests.session()

for i in range(20):
    print('當前爬取餐廳頁面： ' + url)
    res = ss.get(url,headers=headers)
    soup=BeautifulSoup(res.text, 'html.parser')

    for restTags in soup.select('a[class="_15_ydu6b"]'):
        if len(restTags.text.split('. ',1))==1: # 判斷是否為贊助商，是則跳過
            continue
        else:
            restName = restTags.text.split('. ',1)[1] # 在url外獲取餐廳名稱，防止出現簡體字
            
        try:
            restUrl='https://www.tripadvisor.com.tw'+restTags['href']            
            k=1 # 記錄評論頁數
            
            while k!=0: # 進入餐廳網頁後，依次爬取每一頁的評論
                print('嘗試載入第{}頁'.format(k))
                print(restName)
                time.sleep(random.randint(0, 2))
                
                try:
                    ss = requests.session()
                    restRes = ss.get(restUrl,headers=headers)
                    restSoup = BeautifulSoup(restRes.text, 'html.parser')
                    for commentList in restSoup.select('div[class="rev_wrap ui_columns is-multiline"]'):
                        commenter = commentList.select('div[class="info_text pointer_cursor"]')[0].text
                        commentRating = commentList.select('div[class="ui_column is-9"] span')[0]['class'][1].split('bubble_')[-1]
                        commentTitle = commentList.select('span[class="noQuotes"]')[0].text
                        comment = commentList.select('p[class="partial_entry"]')[0].text
#                         print(restName)
                        print(commenter)
#                         print(str(int(commentRating)/10))
#                         print(commentTitle)
#                         print(comment)
#                         print('------')
                        with open('./restComments1_csv/comments.csv', 'a', newline='', encoding='utf_8_sig') as csv_file:
                            fieldnames = ['餐廳名稱','評論賬號','評論分數','評論標題','評論內容']
                            csv_w = csv.DictWriter(csv_file, fieldnames=fieldnames, dialect='excel')
                            csv_w.writerow({'餐廳名稱': restName,\
                                            '評論賬號': commenter,\
                                            '評論分數': str(int(commentRating)/10),\
                                            '評論標題': commentTitle,\
                                            '評論內容': comment})
#                         with open('./restComments1/{}.txt'.format(restName), 'a', encoding='utf-8') as f:
#                                 f.write('評論帳號: '+commenter+'\n')
#                                 f.write('評論分數: '+str(int(commentRating)/10)+'\n')
#                                 f.write('評論標題: '+commentTitle+'\n')
#                                 f.write('評論內容: '+comment+'\n')
                    print('當前評論網址： ' + restUrl)
                    print('載入成功，' + restName + '留言第' + str(k) + '頁***')
                    print('==========================')
                    k=k+1
                    newRestUrl = 'https://www.tripadvisor.com.tw/' + restSoup.select('div[class="unified ui_pagination"] a')[1]['href']
                    
#                     newRestUrl='https://www.tripadvisor.com.tw' + restSoup.select('a[class="nav next ui_button primary"]')[0]['href']
#                     更改前取用的標籤，爬一定數量後會找不到
                    restUrl=newRestUrl
                    print(restUrl)
                    time.sleep(random.randint(1, 3)) # 每爬完一頁評論，休息一次
                except:
                    k=0
                    print(restName + 'has no more comments')
                    print('==========================')
        except IndexError:
            print(IndexError)
            break
    m += 1
    newUrl = 'https://www.tripadvisor.com.tw/RestaurantSearch-g293913-oa{}-Taipei.html'.format(30*m)
    url=newUrl
    print('即將爬取下一頁餐廳網址： ' + url)
    print('=============================')